# Configuração de Ambiente dbt com DuckDB

Este notebook automatiza a criação e configuração de um ambiente `dbt` com `DuckDB` usando ambiente virtual Python.

**Etapas:**
1. Criação de ambiente virtual
2. Instalação de dependências
3. Criação de banco DuckDB
4. Inicialização de projeto dbt
5. Criação do arquivo de perfil do dbt
6. Teste final da configuração

## 0- Imports gerais

In [1]:
%pip install -q duckdb
%pip install -q dbt-duckdb

Note: you may need to restart the kernel to use updated packages.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import subprocess
import venv
import json
from pathlib import Path
import duckdb

## 1. Criação de ambiente virtual

In [3]:
def create_and_activate_venv():
    venv_name = ".venv"
    if os.path.exists(venv_name):
        print(f"✅ Ambiente virtual '{venv_name}' já existe.")
    else:
        print(f"🔧 Criando ambiente virtual '{venv_name}'...")
        venv.create(venv_name, with_pip=True)
        print("✅ Ambiente virtual criado com sucesso!")

    if sys.platform == "win32":
        python_path = os.path.join(venv_name, "Scripts", "python.exe")
        pip_path = os.path.join(venv_name, "Scripts", "pip.exe")
    else:
        python_path = os.path.join(venv_name, "bin", "python")
        pip_path = os.path.join(venv_name, "bin", "pip")
    return python_path, pip_path

python_path, pip_path = create_and_activate_venv()

✅ Ambiente virtual '.venv' já existe.


## 2. Instalação de dependências

In [4]:
def install_packages(pip_path):
    packages = ["dbt-duckdb", "pandas", "duckdb"]
    for package in packages:
        print(f"📦 Instalando {package}...")
        subprocess.run([pip_path, "install", package], check=True)

install_packages(pip_path)

📦 Instalando dbt-duckdb...



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: /workspaces/Atv_DBT_Introducao/.venv/bin/python -m pip install --upgrade pip


📦 Instalando pandas...



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: /workspaces/Atv_DBT_Introducao/.venv/bin/python -m pip install --upgrade pip


📦 Instalando duckdb...



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: /workspaces/Atv_DBT_Introducao/.venv/bin/python -m pip install --upgrade pip


### 3. Inicialização de projeto dbt

```
dbt init meu_projeto_dbt
```

In [5]:
def create_model_directories():
    base_path = "meu_projeto_dbt/models"
    directories = ["raw", "staging", "intermediate", "mart"]
    for d in directories:
        dir_path = os.path.join(base_path, d)
        os.makedirs(dir_path, exist_ok=True)
        init_file = os.path.join(dir_path, "__init__.py")
        if not os.path.exists(init_file):
            with open(init_file, 'w') as f:
                f.write(f"# {d} models\n")
create_model_directories()

### 4. Inicialização de projeto dbt

In [6]:

def create_duckdb_database():
    db_path = "bd/dev.duckdb"
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    conn = duckdb.connect(db_path)
    result = conn.execute("SELECT 'Conexão estabelecida com sucesso!' as status").fetchone()
    print(f"🗄️ Banco DuckDB criado/verificado: {db_path}")
    print(f"📊 Status: {result[0]}")
    conn.close()
    return db_path
    # Criação dos schemas no banco DuckDB
  

create_duckdb_database()

🗄️ Banco DuckDB criado/verificado: bd/dev.duckdb
📊 Status: Conexão estabelecida com sucesso!


'bd/dev.duckdb'

#### 4.1 Teste de conexão ao banco

In [7]:
def test_database_connection():
    conn = duckdb.connect("bd/dev.duckdb")
    result = conn.execute("SELECT version() as duckdb_version").fetchone()
    print(f"🔌 Conexão OK - Versão DuckDB: {result[0]}")
    conn.execute("CREATE TABLE IF NOT EXISTS teste_conexao (id INTEGER, mensagem VARCHAR)")
    conn.execute("INSERT INTO teste_conexao VALUES (1, 'Teste ok')")
    print(conn.execute("SELECT * FROM teste_conexao").fetchall())
    conn.execute("DROP TABLE teste_conexao")
    conn.close()
test_database_connection()

🔌 Conexão OK - Versão DuckDB: v1.4.2
[(1, 'Teste ok')]


###  5. Criação do arquivo de perfil do dbt

In [8]:
try:
    import yaml
except ImportError:
    yaml = None

def create_dbt_profile():
    profile_dir = os.path.expanduser("~/.dbt")
    os.makedirs(profile_dir, exist_ok=True)
    profiles_path = os.path.join(profile_dir, "profiles.yml")
    config = {
        "meu_projeto_dbt": {
            "target": "dev",
            "outputs": {
                "dev": {
                    "type": "duckdb",
                    "path": os.path.abspath("bd/dev.duckdb"),
                    "schema": "main"
                }
            }
        }
    }
    with open(profiles_path, 'w') as f:
        if yaml:
            yaml.dump(config, f)
        else:
            json.dump(config, f, indent=2)
    print(f"✅ Perfil dbt criado em: {profiles_path}")
create_dbt_profile()

✅ Perfil dbt criado em: /home/codespace/.dbt/profiles.yml


### 6. Teste final da configuração

In [9]:
def test_dbt_configuration():
    os.chdir("meu_projeto_dbt")
    dbt_path = os.path.join("..", ".venv", "Scripts" if sys.platform == "win32" else "bin", "dbt")
    result = subprocess.run([dbt_path, "debug"], capture_output=True, text=True)
    print(result.stdout)
    os.chdir("..")
test_dbt_configuration()

18:09:16  Running with dbt=1.10.15
18:09:16  dbt version: 1.10.15
18:09:16  python version: 3.12.3
18:09:16  python path: /workspaces/Atv_DBT_Introducao/.venv/bin/python
18:09:16  os info: Linux-6.8.0-1030-azure-x86_64-with-glibc2.39
18:09:18  Using profiles dir at /workspaces/Atv_DBT_Introducao/meu_projeto_dbt
18:09:18  Using profiles.yml file at /workspaces/Atv_DBT_Introducao/meu_projeto_dbt/profiles.yml
18:09:18  Using dbt_project.yml file at /workspaces/Atv_DBT_Introducao/meu_projeto_dbt/dbt_project.yml
18:09:18  adapter type: duckdb
18:09:18  adapter version: 1.10.0
18:09:19  Configuration:
18:09:19    profiles.yml file [OK found and valid]
18:09:19    dbt_project.yml file [OK found and valid]
18:09:19  Required dependencies:
18:09:19   - git [OK found]

18:09:19  Connection:
18:09:19    database: dev
18:09:19    schema: main
18:09:19    path: ../bd/dev.duckdb
18:09:19    config_options: None
18:09:19    extensions: None
18:09:19    settings: {}
18:09:19    external_root: .
18:09: